# Exploratory Data Analytics (EDA) Retail

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pandas_profiling import ProfileReport

C:\Users\KuAn\AppData\Local\Temp\ipykernel_19884\2324422676.py:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load data
df_products = pd.read_csv('../data/Products_with_Categories.csv')
df_products.head()

,productId,productName,price,Category
0,1,tropical fruit,7.8,Fresh Food
1,2,whole milk,1.8,Dairy
2,3,pip fruit,3.0,Fresh Food
3,4,other vegetables,0.8,Fresh Food
4,5,rolls/buns,1.2,Bakery & Sweets


In [4]:
# Profiling report
profile = ProfileReport(df_products, title="Pandas Profiling Report", explorative=True)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10.34it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   productId    167 non-null    int64  
 1   productName  167 non-null    object 
 2   price        167 non-null    float64
 3   Category     167 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 5.3+ KB


In [6]:
df_transactions = pd.read_csv('../data/Transactions.csv')
df_transactions.head()

,Member_number,Date,productId,items
0,1808,21-07-2015,1,3
1,2552,05-01-2015,2,1
2,2300,19-09-2015,3,3
3,1187,12-12-2015,4,3
4,3037,01-02-2015,2,1


In [7]:
# Merge the df_products and df_transactions
df = pd.merge(df_products, df_transactions, on='productId', how='outer')
display(df.head())

,productId,productName,price,Category,Member_number,Date,items
0,1,tropical fruit,7.8,Fresh Food,1808,21-07-2015,3
1,1,tropical fruit,7.8,Fresh Food,4119,12-02-2015,3
2,1,tropical fruit,7.8,Fresh Food,3962,18-09-2015,2
3,1,tropical fruit,7.8,Fresh Food,1088,30-11-2015,2
4,1,tropical fruit,7.8,Fresh Food,1863,04-08-2015,1


In [8]:
df.shape

(38765, 7)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   productId      38765 non-null  int64  
 1   productName    38765 non-null  object 
 2   price          38765 non-null  float64
 3   Category       38765 non-null  object 
 4   Member_number  38765 non-null  int64  
 5   Date           38765 non-null  object 
 6   items          38765 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.1+ MB


In [10]:
df.describe()

,productId,price,Member_number,items
count,38765.000000,38765.000000,38765.000000,38765.000000
mean,39.956094,4.295170,3003.641868,1.996131
std,37.294166,4.313656,1153.611031,0.817645
min,1.000000,0.300000,1000.000000,1.000000
25%,9.000000,1.250000,2002.000000,1.000000
50%,25.000000,2.500000,3005.000000,2.000000
75%,64.000000,6.100000,4007.000000,3.000000
max,167.000000,28.500000,5000.000000,3.000000


In [11]:
# Check null
df.isnull().sum()

productId        0
productName      0
price            0
Category         0
Member_number    0
Date             0
items            0
dtype: int64

In [12]:
# Convert 'Date' to datetime objects
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

In [13]:
# Convert 'Date' column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])
# Calculate Recency
df['Total_Revenue'] = df['items'] * df['price']
df.head()

,productId,productName,price,Category,Member_number,Date,items,Total_Revenue
0,1,tropical fruit,7.8,Fresh Food,1808,2015-07-21,3,23.4
1,1,tropical fruit,7.8,Fresh Food,4119,2015-02-12,3,23.4
2,1,tropical fruit,7.8,Fresh Food,3962,2015-09-18,2,15.6
3,1,tropical fruit,7.8,Fresh Food,1088,2015-11-30,2,15.6
4,1,tropical fruit,7.8,Fresh Food,1863,2015-08-04,1,7.8


In [14]:
# Check outliers

def find_outliers_iqr(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return outliers

# Create figure and axes
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Iterate through columns to check
for i, col in enumerate(['price', 'Total_Revenue']):
    # Identify outliers using IQR
    outliers = find_outliers_iqr(df[col])
    
    # Calculate the percentage of outliers
    outlier_percent = len(outliers) / len(df) * 100
    
    # Plot box plot
    sns.boxplot(y=df[col], ax=axes[i], color='skyblue')
    axes[i].set_title(f'{col}')
    
    # Display outlier percentage
    axes[i].text(0.95, 0.95, f'{outlier_percent:.2f}% outliers', 
                 transform=axes[i].transAxes, ha='right', va='top')

plt.tight_layout()
plt.show()

1. Price:

Right skewed distribution: Some products are priced significantly higher than the majority of other products.
The median is low, indicating that the majority of products are priced low.
The outlier (4.11%), indicating that there are a significant number of products with unusually high prices.

3. Total_Revenue:
Right skewed distribution: Similar to price, total revenue also has a right skewed distribution, with a long tail on the right side. This indicates that there are some transactions with very high revenue.
Low median: The median of total revenue is low, indicating that the majority of transactions have low revenue.
Many outliers: The chart shows 8.61% outliers, indicating that there are a significant number of transactions with unusually high revenue.
Conclusion:

Price and Revenue Variance: There is significant price and revenue variation across transactions, with some transactions having unusually high values.

Consistent Purchasing Behavior: The number of items purchased in each transaction appears consistent, suggesting that customers typically purchase a fixed number of items.

In [15]:
min_day = df['Date'].min()
max_day = df['Date'].max()

print("Min day:", min_day)
print("Max day:", max_day)

Min day: 2014-01-01 00:00:00
Max day: 2015-12-30 00:00:00


In [16]:
# Calculate the frequency count of items (number of times each item appears in the data)
item_frequency = df['items'].value_counts()
item_frequency.sort_values(ascending=False)

items
1    13033
3    12883
2    12849
Name: count, dtype: int64

In [17]:
# Number of unique products
num_unique_products = df['productName'].nunique()
print("Number of unique products:", num_unique_products)

Number of unique products: 167


In [18]:
# Number of unique customers
num_unique_customers = df['Member_number'].nunique()
print("Number of unique customers:", num_unique_customers)

Number of unique customers: 3898


In [19]:
# Calculate total revenue per customer
customer_revenue = df.groupby('Member_number')['Total_Revenue'].sum().sort_values(ascending=False)
customer_revenue.head(10)

Member_number
2433    375.31
2193    361.45
1793    345.10
3289    334.15
2743    312.46
2811    305.72
2990    303.61
3872    301.33
4753    301.10
2993    296.96
Name: Total_Revenue, dtype: float64

In [20]:
# Create a summary table for categories
category_summary = df['Category'].value_counts().reset_index()
category_summary.columns = ['Category', 'Frequency']

# Calculate and round the percentage
category_summary['Percentage'] = round((category_summary['Frequency'] / category_summary['Frequency'].sum()) * 100, 2)

# Display the table
category_summary

,Category,Frequency,Percentage
0,Fresh Food,11501,29.67
1,Dairy,7554,19.49
2,Beverages,6044,15.59
3,Bakery & Sweets,5962,15.38
4,Household & Hygiene,2177,5.62
5,Frozen & Processed Food,1818,4.69
6,Pantry Staples,1560,4.02
7,Specialty & Seasonal,1190,3.07
8,Snacks,473,1.22
9,Pet Care,329,0.85


In [21]:
# 1. Category Analysis
category_counts = df['Category'].value_counts()
plt.figure(figsize=(10, 6))
category_counts.plot(kind='bar')
plt.title('Frequency of Each Category')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [22]:
# 2. Product and Category Sales Analysis
# --- Plot 1: Top 10 products by revenue ---
product_sales = df.groupby('productName')[['Total_Revenue', 'items']].sum().sort_values('Total_Revenue', ascending=False)
top_10_products = product_sales.head(10).copy()
top_10_products['Category'] = top_10_products.index.map(df.set_index('productName')['Category'].to_dict())

plt.figure(figsize=(12, 6))
bars = plt.bar(top_10_products.index, top_10_products['Total_Revenue'], color='skyblue', edgecolor='black')  
plt.xlabel('Product Name', fontsize=12)  
plt.ylabel('Total Revenue', fontsize=12)  
plt.title('Top 10 Products with Highest Revenue', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)  
plt.grid(axis='y', linestyle='--', alpha=0.5)  
for bar, category in zip(bars, top_10_products['Category']):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.1, f'{yval:,.0f}\n({category})', ha='center', va='bottom', fontsize=10) 
plt.tight_layout()
plt.show()

# --- Plot 2: Top 10 products by customer count ---
top_products = df.groupby(['productName', 'Category'])['Member_number'].count().reset_index()
top_products = top_products.sort_values('Member_number', ascending=False)[:10]

plt.figure(figsize=(12, 6))
bars = plt.bar(top_products['productName'], top_products['Member_number'], color='skyblue', edgecolor='black')
for bar, category in zip(bars, top_products['Category']):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.1, f'{yval:,.0f}\n({category})', ha='center', va='bottom', fontsize=10)
plt.title('Top 10 Most Purchased Products by Customers', fontsize=16, fontweight='bold')
plt.xlabel('Product', fontsize=12)
plt.ylabel('Customer Count', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


Chart 1: Top 10 Products with the Highest Revenue

- Revenue is concentrated in a few products: The chart shows that revenue is not evenly distributed among products. There are some products with revenue that is superior to the rest.

Fresh Food is the product group with the highest revenue, clearly dominating over other groups.

- Diversity in product groups: Although fresh food has the highest revenue, other product groups such as Household & Hygiene, Dairy, Bakery & Sweets also contribute significantly to revenue.

- Meat products (Beef, Pork, Hamburger Meat) have good revenue: This shows that customers' demand for meat consumption is quite high.

Chart 2: Top 10 Most Purchased Products

- Dairy products are popular: Whole milk is the most purchased product by customers. Other dairy products such as yogurt are also in the top 10.

- Other Vegetables, Root Vegetables, Tropical Fruit, Citrus Fruit have good purchasing power: This shows that customers are interested in fresh and healthy products.

- Bread & Sweets (Rolls/Buns, Bakery & Sweets) are purchased a lot: This product group shows that customers are also interested in fast and convenient products.

- Beverages (Soda, Bottled Water, Beverages) have stable purchasing power: This product group shows that customers' thirst quenching demand is quite large.

Relationship between the two charts:

- Difference between revenue and purchase quantity: Some products have high revenue but are not purchased much (e.g. beef), while some products are purchased much but have low revenue (e.g. whole milk). This shows the difference in price and order value between products.

- Fresh Food is an important product group: This product group appears in both charts, showing that this is a product group that has a large impact on revenue and is popular with many customers.

In [23]:
# Group transactions by the number of items purchased
item_freq_table = df['items'].value_counts().reset_index()
item_freq_table.columns = ['Number of Items', 'Number of Transactions']

# Sort the table by the number of items
item_freq_table = item_freq_table.sort_values(by='Number of Items')

# Display the table
print(item_freq_table)

plt.figure(figsize=(10, 5))
sns.histplot(df['items'], bins=30, kde=True, color='blue')
plt.xlabel('Number of items per transaction')
plt.ylabel('Frequency')
plt.title('Distribution of Items Purchased per Transaction')
plt.show()

   Number of Items  Number of Transactions
0                1                   13033
2                2                   12849
1                3                   12883


In [24]:
# 3. Distribution of Price, Items, and Total Revenue
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 12))

for i, col in enumerate(['price', 'items', 'Total_Revenue']):
    sns.histplot(df[col], kde=True, ax=axes[i, 0], label=col, color='blue')
    axes[i, 0].set_title(f'Distribution of {col}')
    axes[i, 0].legend()
    
    sns.boxplot(y=df[col], ax=axes[i, 1], color='orange')
    axes[i, 1].set_title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()


The charts show that customers' shopping behavior is usually to buy a few items (1-3) with low or medium values. However, there are also some larger purchases with higher values.

Correlation: There is a correlation between "price" and "Total Revenue", as both have right-skewed distributions and have many outliers at the top. However, "items" has a different distribution.

1. Price:

Multi-peaked distribution: The distribution chart shows that "price" has many peaks, indicating that there are many common value groups. This may indicate that the products sold have many different price ranges.

Right-skewed: The Box Plot shows that the data is right-skewed, with some high outliers.

Range: Most of the values ​​are concentrated from 0 to around 4-5, then there are smaller peaks at higher value ranges.
Outlier: The Box Plot confirms the presence of high outliers.

2. Items:

Sparse distribution: "items" only take on 3 values ​​1, 2 and 3, indicating that transactions usually purchase only a few items.
Multi-peaked distribution: The distribution shows 3 distinct peaks at 1, 2 and 3, indicating that transactions usually purchase exactly 1, 2 or 3 items.
Outlier: The Box Plot shows no significant outliers.

3. Total Revenue:

Right-skewed: The Box Plot shows that "Total Revenue" is right-skewed, with many low values ​​and a few high values.
Multi-peaked distribution: The distribution shows multiple peaks, indicating that there are many groups of transactions with different revenue levels.
Outlier: The Box Plot shows many high outliers, indicating that there are some transactions with very large revenues.
Range: Most of the values ​​are concentrated in the range of 0 to 5, then there are smaller peaks at higher ranges.

In [25]:
# 4. Customer Analysis
category_customers = df.groupby('Category')['Member_number'].nunique().sort_values(ascending=False)
print("\nNumber of Customers by Category:\n", category_customers)

top_10_customers = df.groupby('Member_number')['items'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_10_customers.index, y=top_10_customers.values)
plt.title('Top 10 Customers by Total Items Purchased')
plt.xlabel('Member Number')
plt.ylabel('Total Items Purchased')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


Number of Customers by Category:
 Category
Fresh Food                 3538
Dairy                      3130
Beverages                  2925
Bakery & Sweets            2874
Household & Hygiene        1615
Frozen & Processed Food    1421
Pantry Staples             1275
Specialty & Seasonal        993
Snacks                      439
Pet Care                    315
Personal Care               152
Name: Member_number, dtype: int64


In [26]:
#5 Calculate total revenue for each Member_number
customer_revenue = df.groupby('Member_number')['Total_Revenue'].sum().reset_index()

# Plot scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Member_number', y='Total_Revenue', data=customer_revenue)
plt.title('Relationship between Member_number and Total_Revenue')
plt.xlabel('Member Number')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels if needed
plt.tight_layout()
plt.show()


- Wide Dispersion: The data points are widely dispersed across the chart, indicating that there is no clear linear relationship between the number of members (Member_number) and total revenue (Total_Revenue).
- Concentration at low revenue levels: The majority of the data points are concentrated at low revenue levels (below 100), indicating that many members have low total revenue.
Outlier: There are a few data points with very high revenue (above 300), indicating that there are some members who contribute significantly to revenue.

In [27]:
# 6. Monthly Sales Analysis
df['month'] = df['Date'].dt.month
df_months = df.groupby('month').agg({'Total_Revenue': 'sum'}).reset_index()

# Tạo bảng doanh thu theo tháng
print("Bảng doanh thu theo tháng:")
print(df_months)

sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_months, x='month', y='Total_Revenue', marker='o', color='darkblue', linewidth=2)
plt.title('Total Sales by Month', fontsize=16)
plt.xlabel('Month of the Year', fontsize=12)
plt.ylabel('Total Revenue', fontsize=12)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

Bảng doanh thu theo tháng:
    month  Total_Revenue
0       1       29449.42
1       2       25968.07
2       3       27083.45
3       4       27889.51
4       5       29599.98
5       6       27746.55
6       7       28609.30
7       8       30664.29
8       9       25923.77
9      10       27660.92
10     11       26786.40
11     12       24777.97


Biến động lớn: Doanh số có sự biến động rất lớn giữa các tháng, cho thấy sự không ổn định trong hoạt động kinh doanh.
Tháng có doanh số cao nhất: Tháng 5 có doanh số cao nhất, tiếp theo là tháng 8. Điều này có thể do các chương trình khuyến mãi, sự kiện đặc biệt hoặc yếu tố mùa vụ.

Tháng có doanh số thấp nhất: Tháng 2 và tháng 12 có doanh số thấp nhất. Điều này có thể do các yếu tố như kỳ nghỉ lễ dài, thời tiết xấu hoặc giảm nhu cầu tiêu dùng.

Xu hướng tăng giảm: Doanh số có xu hướng tăng từ tháng 2 đến tháng 5, sau đó giảm mạnh vào tháng 6, tăng trở lại vào tháng 8 và giảm dần đến cuối năm.

In [28]:
# 7. Daily Sales Analysis
df['weekday'] = df['Date'].dt.day_name()
daywise = df.groupby('weekday').agg({'Total_Revenue': 'sum'}).reset_index()
daywise['weekday'] = pd.Categorical(daywise['weekday'], categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True)
daywise = daywise.sort_values('weekday')

plt.figure(figsize=(10, 6))
plt.plot(daywise['weekday'], daywise['Total_Revenue'], marker='o', color='skyblue', linewidth=1.2, markersize=6)
plt.title('Total Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.show()


Ngày trong tuần có doanh số cao nhất: Doanh số đạt đỉnh điểm vào thứ Tư và thứ Năm. Điều này cho thấy có thể có những chương trình khuyến mãi hoặc hoạt động đặc biệt nào đó vào giữa tuần, thu hút nhiều khách hàng hơn.

Ngày cuối tuần có doanh số thấp nhất: Doanh số giảm mạnh vào thứ Bảy và Chủ Nhật. Điều này có thể là do khách hàng có xu hướng dành thời gian cho các hoạt động khác vào cuối tuần thay vì mua sắm.

Sự biến động: Doanh số có sự biến động khá lớn giữa các ngày trong tuần, cho thấy sự khác biệt rõ rệt trong hành vi mua sắm của khách hàng.

In [29]:
# Save
df.to_csv('../data/rfm_data.csv', index=False)